In [1]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig,RunnableGenerator
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf
from typing import Iterator,List,AsyncIterator

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [5]:
# llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llm=ChatOpenAI(model="gpt-3.5-turbo")

In [6]:
def lengthFunction(text:str):
    return len(text)

def _multipleLengthFunction(text1:str,text2:str):
    return len(text1)*len(text2)

def multipleLengthFunction(_dict:dict):
    return _multipleLengthFunction(text1=_dict['text1'],text2=_dict['text2'])

In [7]:
prompt=ChatPromptTemplate.from_template(template="What is {a}+{b}")

In [8]:
chain=(
    {
        "a":itemgetter("foo")|RunnableLambda(func=lengthFunction),
        "b":
            {
                "text1":itemgetter("foo"),
                "text2":itemgetter("bar")
            } | RunnableLambda(func=multipleLengthFunction)
    } 
    | prompt 
    | llm 
    | StrOutputParser()
)


In [9]:
chain.invoke(input={"foo":"bar","bar":"ritz"})

'3 + 12 = 15'

<h3>Using Lambda in Streaming</h3>

In [10]:
prompt=ChatPromptTemplate.from_template(
    template="Write a comma-separated list of 5 animals similar to: {animal}. Do not include numbers or bullets, spaces or newlines"
)

In [11]:
strChain={'animal':RunnablePassthrough()} |prompt | llm |StrOutputParser()

In [12]:
strChain.invoke(input="Snake")

'Lizard, Turtle, Crocodile, Worm, Salamander'

In [13]:
strChain=prompt | llm |StrOutputParser()
strChain.invoke(input={"animal":"Snake"})

'Eel, Python, Anaconda, Cobra, Rattlesnake'

In [14]:
for chunk in strChain.stream(input="Snake"):
    print(chunk,flush=True,end="|")

|L|izard|,| Worm|,| E|el|,| Ana|conda|,| Python||

In [19]:
## Custom Parser that splits an iterator of llm tokens into a list of strings separated by commas
# The entire chunk is generated and passed through the Runnable Lambda

def splitIntoList(input:Iterator[str]) -> Iterator[list[str]]:
    # Hold Partial Input, until we get a Comma
    buffer=""
    for chunk in input:  # Chunks are actually the individual letters for stream
        # print(chunk)  # to be commented
        buffer+=chunk
        while "," in buffer:
            commaIndex=buffer.index(",")
            yield [buffer[:commaIndex].strip()]
            if commaIndex==len(buffer)-1:
                buffer=""
            else:
                buffer=buffer[commaIndex+1:].strip()
    yield [buffer.strip()]

In [20]:
listChain={'animal':RunnablePassthrough()} |prompt | llm |StrOutputParser()|RunnableLambda(func=splitIntoList)

In [21]:
for chunk in listChain.stream(input="snake"):
    print(chunk,flush=True)

['python']
['cobra']
['boa']
['viper']
['anaconda']


In [22]:
for chunk in listChain.stream(input="wife"):
    print(chunk,flush=True)

['husband']
['partner']
['spouse']
['mate']
['companion']


#### Async Iteration

In [48]:
async def asplitIntoList(input:AsyncIterator[str]) -> AsyncIterator[List[str]]:
    # Hold Partial Input, until we get a Comma
    buffer=""
    async for chunk in input:  # Chunks are actually the individual letters for stream
        # print(chunk)  # to be commented
        buffer+=chunk
        while "," in buffer:
            commaIndex=buffer.index(",")
            yield [buffer[:commaIndex].strip()]
            if commaIndex==len(buffer)-1:
                buffer=""
            else:
                buffer=buffer[commaIndex+1:].strip()
    yield [buffer.strip()]

In [49]:
alistChain={'animal':RunnablePassthrough()} |prompt | llm |StrOutputParser()|RunnableGenerator(transform=asplitIntoList)

In [50]:
async for chunk in alistChain.astream(input="tiger"):
    print(chunk,flush=True)

['lion']
['leopard']
['jaguar']
['cheetah']
['panther']
